In [1]:
require 'optim'
require 'io'
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

In [70]:
data_path = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/'

In [79]:
dofile("utils.lua")
dofile("model_utils.lua")

...Utils file loaded	


In [72]:
inputs = {
        ['inputs'] = '2012_aurora_shooting_first_sentence_numtext2.csv', 
        ['nuggets'] = 'aurora_nuggets_numtext.csv'
}
query_id = 1

In [82]:
K_tokens = 25

In [83]:
input_file = csvigo.load({path = data_path .. inputs['inputs'], mode = "large", verbose = false})
input_file = geti_n(input_file, 2, #input_file) 

In [84]:
xtdm  = buildTermDocumentTable(input_file, K_tokens)

In [89]:
xs = torch.LongTensor(padZeros(xtdm, K_tokens))

In [91]:
nugget_file = csvigo.load({path = data_path .. inputs['nuggets'], mode = "large", verbose = false})

In [97]:
nugget_file = geti_n(nugget_file, 2, #nugget_file) 
nuggets = buildTermDocumentTable(nugget_file, nil)
K_nuggs = getMaxseq(nugget_file)

In [105]:
out = {}
for i=1, #nuggets do
    out = tableConcat(table.unpack(nuggets), out)
end

In [108]:
mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(actionsSize, sentencesSize)

In [2]:
x = torch.rand(10, 2)

In [7]:
x

 0.8994  0.6371
 0.3149  0.9264
 0.8530  0.6391
 0.8876  0.2793
 0.6963  0.7829
 0.9394  0.9384
 0.8438  0.2985
 0.4222  0.0262
 0.5286  0.6770
 0.9841  0.4796
[torch.DoubleTensor of size 10x2]



In [6]:
x[{{1, 3}}]

 0.8994  0.6371
 0.3149  0.9264
 0.8530  0.6391
[torch.DoubleTensor of size 3x2]



In [10]:
x[{2}]

 0.3149
 0.9264
[torch.DoubleTensor of size 2]

